In [1]:
import pandas as pd
import numpy as np
import configparser

In [2]:
!pip install haversine

In [2]:
from azureml.core import Datastore,Workspace,Dataset
# Get a named datastore from the current workspace
ws = Workspace.from_config()
datastore = Datastore.get(ws, datastore_name='aisdatastore')

In [3]:
rotte_dataset = ws.datasets["rotte"]

In [4]:
rotte_dataset = rotte_dataset.to_pandas_dataframe()

Resolving access token for scope "https://datalake.azure.net//.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=clientid) and endpoint type based on configuration


In [5]:
ais_dataset =  ws.datasets["ais"]

In [6]:
ais_dataset = ais_dataset.to_pandas_dataframe()

In [8]:
#config = configparser.ConfigParser()
#config.read('ais_config.ini')

In [9]:
# read values from a section
#ais_dataset_name = config.get('INIT', 'ais_dataset_name')
#rotte_dataset_name = config.get('INIT', 'rotte_dataset_name')
#rottesort_filename = config.get('PREPROCESS', 'rottesort_filename')

#world_porti_filename = config.get('INIT', 'world_porti_filename')
#arrivi_nei_porti_fname = config.get('ELABORATION','arrivi_nei_porti_fname')
#rotte_arrivi_porti_world_fname = config.get('ELABORATION','rotte_arrivi_porti_world_fname')
#rotte_arrivi_porti_all_fname = config.get('ELABORATION','rotte_arrivi_porti_all_fname')

#ais_dataset = pd.read_csv(ais_dataset_name, sep=',')
#ais_dataset.head()

In [10]:
vessels_type = ais_dataset['type_summary'].unique()
vessels_flag = ais_dataset['flag'].unique()

In [11]:
print(vessels_type)
print(vessels_flag)

['Cargo' 'Tug' 'Pleasure Craft' 'NULL' 'Passenger' 'Tanker'
 'Special Craft' 'High Speed Craft' 'Fishing' 'Sailing Vessel'
 'Search and Rescue' 'Other' 'Unspecified']
['--' 'KY' 'IT' 'GB' 'DZ' 'MH' 'AL' 'BE' 'BG' 'CY' 'DE' 'MD' 'MT' 'DK'
 'ES' 'FR' 'GI' 'GR' 'HR' 'MA' 'NL' 'PA' 'KI' 'RU' 'BS' 'LU' 'MC' 'PT'
 'NO' 'SE' 'CH' 'TR' 'UA' 'LV' 'LT' 'AG' 'BM' 'BZ' 'BB' 'HN' 'US' 'JM'
 'KN' 'VC' 'VG' 'SA' 'CN' 'TW' 'AZ' 'IL' 'JP' 'KZ' 'KR' 'KW' 'LB' 'QA'
 'HK' 'PW' 'CK' 'NU' 'PH' 'SG' 'TV' 'VN' 'VU' 'WF' 'CM' 'KM' 'EG' 'LR'
 'PL' 'LY' 'SL' 'TG' 'TN']


In [12]:
vessels_mmsi = ais_dataset['mmsi'].unique().astype(int)
vessels_imo = ais_dataset['imo'].unique().astype(int)

print(len(vessels_mmsi))
print(len(vessels_imo))

3192
3164


In [13]:
world_porti = ws.datasets["porti1"]

In [14]:
try:
    world_porti = world_porti.to_pandas_dataframe()
except Exception as e:
    print(e)


In [15]:
#rotte_dataset = pd.read_csv(rotte_dataset_name, sep=',', low_memory=False)

In [15]:
#drop NA values
rotte_dataset=rotte_dataset.dropna()

In [16]:
#build timestamp column to be added to rotte_092021 dataframe
stamp=rotte_dataset['stamp'].astype(float)
#stamp
from datetime import datetime
timestamp_column = stamp.apply(lambda x: datetime.fromtimestamp(x))
#timestamp_column

In [17]:
#add timestamp column to rotte_092021 dataframe
rotte_dataset['timestamp'] = timestamp_column.values

/tmp/ipykernel_7678/2719174265.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rotte_dataset['timestamp'] = timestamp_column.values


In [19]:
#rotte_dataset.head()

,mmsi,lng,lat,speed,a,b,time,stamp,c,d,timestamp
0,248730000.0,8.371692,40.84104,0.0,115.0,3.0,2021-09-01,1.630488e+09,0.0,-1.0,2021-09-01 09:24:50
1,257328000.0,-85.628200,30.13659,0.0,313.0,56.0,2021-09-01,1.630488e+09,0.0,-1.0,2021-09-01 09:24:20
2,248730000.0,8.371658,40.84102,0.0,115.0,3.0,2021-09-01,1.630462e+09,0.0,-1.0,2021-09-01 02:00:35
3,248730000.0,8.371685,40.84101,0.0,115.0,3.0,2021-09-01,1.630461e+09,0.0,-1.0,2021-09-01 01:56:14
4,248730000.0,8.371672,40.84102,0.0,115.0,3.0,2021-09-01,1.630461e+09,0.0,-1.0,2021-09-01 01:51:33


In [18]:
#sort_dataset by mmsi,stamp,lng,lat
rotte_dataset=rotte_dataset.sort_values(by=['mmsi','stamp','lng','lat'],ascending=[True,True,True,True])
#rotte_dataset.head()

In [21]:
#Analysis of data
#rotte_dataset_218158000 = rotte_dataset[rotte_dataset['mmsi']=='218158000']

#rotte_dataset_201100115 = rotte_dataset[rotte_dataset['mmsi']=='201100115']

#rotte_dataset_247046800 = rotte_dataset[rotte_dataset['mmsi']==247046800]
#rotte_dataset_247046800.to_csv("df_rotte_dataset_247046800_set2021.csv",index=False,sep=',')

rotte_dataset_247043300 = rotte_dataset[rotte_dataset['mmsi']==247043300]
rotte_dataset_247043300.to_csv("df_rotte_dataset_247043300_set2021.csv",index=False,sep=',')



In [24]:
ais_dataset[ais_dataset['mmsi']==247043300]


,lat,lng,speed,course,heading,elapsed,destination,flag,length,rot,...,w_left,dwt,gt_shiptype,shiptype,time,imo,mmsi,porto,type_summary,cod_porto
7878,40.84383,14.26333,0.0,NULL,NULL,4,None,IT,1.0,NULL,...,0.0,NULL,NaN,1,2021-09-06,5258482,247043300,Napoli,NULL,IT00ITNAP
7879,40.83452,14.26462,0.0,32,NULL,3,MARE,IT,57.0,NULL,...,6.0,25,23.0,6,2021-09-07,5258482,247043300,Napoli,Passenger,IT00ITNAP
7880,40.83451,14.26461,0.0,27,NULL,493,MARE,IT,57.0,NULL,...,6.0,25,23.0,6,2021-09-08,5258482,247043300,Napoli,Passenger,IT00ITNAP
7881,40.83452,14.26463,0.0,29,NULL,3,MARE,IT,57.0,NULL,...,6.0,25,23.0,6,2021-09-14,5258482,247043300,Napoli,Passenger,IT00ITNAP
7882,40.83451,14.26464,0.0,30,NULL,469,MARE,IT,57.0,NULL,...,6.0,25,23.0,6,2021-09-15,5258482,247043300,Napoli,Passenger,IT00ITNAP
7883,40.84383,14.26333,0.0,NULL,NULL,4,None,IT,1.0,NULL,...,0.0,NULL,NaN,1,2021-09-19,5258482,247043300,Napoli,NULL,IT00ITNAP
7884,40.83512,14.26530,46.0,109,NULL,10,MARE,IT,57.0,NULL,...,6.0,25,23.0,6,2021-09-24,5258482,247043300,Napoli,Passenger,IT00ITNAP
7885,40.84310,14.26370,0.0,109,NULL,822,MARE,IT,57.0,NULL,...,6.0,25,23.0,6,2021-09-25,5258482,247043300,Napoli,Passenger,IT00ITNAP
7886,40.83451,14.26462,0.0,23,NULL,8,MARE,IT,57.0,NULL,...,6.0,25,23.0,6,2021-09-28,5258482,247043300,Napoli,Passenger,IT00ITNAP


In [22]:
#Analysis of data
#rotte_dataset_218158000.to_csv("DCME\\AIS\\rotte_with_speed_2021\\rotte_arrivi_porti_set2021_218158000.csv",index=False,sep=',')
#rotte_dataset_201100115.to_csv("DCME\\AIS\\rotte_with_speed_2021\\rotte_arrivi_porti_set2021_201100115.csv",index=False,sep=',')

In [23]:
class Utils(object):
   def intersect(self, nums1, nums2):
      """
      :type nums1: List[int]
      :type nums2: List[int]
      :rtype: List[int]
      """
      m = {}
      print(len(nums1))
      print(len(nums2))
      if len(nums1)<len(nums2):
         nums1,nums2 = nums2,nums1
      for i in nums1:
         if i not in m:
            m[i] = 1
         else:
            m[i]+=1
      result = []
      for i in nums2:
         if i in m and m[i]:
            m[i]-=1
            result.append(i)
      return result

In [24]:
rotte_dataset_mmsi =  rotte_dataset['mmsi'].unique().astype(int)
print(len(rotte_dataset_mmsi))

2840


In [25]:
utils = Utils()
mmsi_intersection = utils.intersect(vessels_mmsi.tolist(), rotte_dataset_mmsi.tolist())
#print(f'result intersection: {mmsi_intersection}')

3192
2840


In [26]:
#Drop rows whose mmsi is not in rotte_092021_simplified(rotte_092021_mmsi,vessels_mmsi) from rotte_092021,
# i.e. get only rows whos mmsi is in mmsi_intersection

rotte_dataset_simplified = rotte_dataset[rotte_dataset['mmsi'].astype(int).isin(mmsi_intersection)]

#rotte_dataset_simplified = rotte_dataset[rotte_dataset['mmsi'].astype(int).isin(vessels_mmsi)]
#rotte_dataset_simplified.head()
len(rotte_dataset_simplified)

33474627

In [27]:
rotte_dataset_simplified = rotte_dataset_simplified.reset_index()
#rotte_dataset_simplified.head()

ELABORAZIONE DELLE ROTTE

In [28]:
from datetime import datetime
import haversine as hs
from haversine import Unit

In [29]:
#elaborazione degli arrivi (df)
def arrival_elaboration(df_rotte):
    #df_rotte = rottesort

    dim=len(df_rotte)
    print("df_rotte len before: ",dim)

    df_arrival = pd.DataFrame(columns = ['row','mmsi','arrival','departure','lng','lat','lng_orig','lat_orig','speed','status'])

    oldmmsi=0
    sumrec=0
    start=0
    lat_orig=0
    lng_orig=0
    oldlng=0
    oldlat=0
    start = None
    i=0
    status=0 ###0-nuovo 1-arrivato 2-partito
    end = None
    speed = None

    start_time = datetime.now()

    for item in df_rotte.itertuples():
        try:
            #item = df_rotte.iloc[i]
            mmsi,time_voyage,lng,lat,speed = item.mmsi,item.timestamp,float(item.lng),float(item.lat),int(item.speed)

            if(mmsi!=oldmmsi):
                if (status==1):#si riferisce alla old ship
                    df_arrival=df_arrival.append({'row':i,'mmsi':oldmmsi,'arrival':start,
                           'departure':end,'lng':oldlng,'lat':oldlat,'lng_orig':lng_orig,'lat_orig':lat_orig,
                            'speed':speed,'status':status},ignore_index=True)
                start,status,lng_orig,lat_orig,oldlng,oldlat,oldmmsi=None,0,0,0,0,0,mmsi
                oldmmsi = mmsi
    
            if(speed==0):          
                if (status==0):
                    start,oldlng,oldlat = time_voyage,lng,lat
                
                #To calculate distance in meters
                if(status<2):
                    loc1=(lat,lng)
                    loc2=(oldlat,oldlng)
            
                    distance = hs.haversine(loc1,loc2,unit=Unit.METERS)
            
                    if(distance > 3000.0):
                        #print(distance)
                        #if ((abs(oldlng-lng)+abs(oldlat-lat))>0.3):
                        df_arrival=df_arrival.append({'row':i,'mmsi':oldmmsi,'arrival':start,
                           'departure':end,'lng':oldlng,'lat':oldlat,
                                       'lng_orig':lng_orig,'lat_orig':lat_orig,'speed':speed,'status':status},ignore_index=True)
                        start,lng_orig,lat_orig,oldlng,oldlat = time_voyage,oldlng,oldlat,lng,lat
                    
                if(status==2):
                    start,lng_orig,lat_orig,oldlng,oldlat = time_voyage,oldlng,oldlat,lng,lat
                    
                end=time_voyage
                status=1
                
            if (speed>0):
                if(status==1):
                    loc1=(lat,lng)
                    loc2=(oldlat,oldlng)
                    distance = hs.haversine(loc1,loc2,unit=Unit.METERS)
                    if(distance > 3000.0):
                        df_arrival=df_arrival.append({'row':i,'mmsi':oldmmsi,'arrival':start,
                           'departure':end,'lng':oldlng,'lat':oldlat,
                            'lng_orig':lng_orig,'lat_orig':lat_orig,'speed':speed,'status':2},ignore_index=True)
                        
                        status,lng_orig,lat_orig = 2,oldlng,oldlat
                if(status==0):
                    status=2
                   
            #stampa di controllo
            if(i%1000000 == 0):
               print(i)
        except Exception as e:
            print(i," ")
            print(e)
        i+=1
    #end for
    
    #scrive l'ultimo record
    if(status==1):
            df_arrival=df_arrival.append({'row':i,'mmsi':oldmmsi,'arrival':start,
                        'departure':end,'lng':oldlng,'lat':oldlat,
                        'lng_orig':lng_orig,'lat_orig':lat_orig,'speed':speed,'status':status},ignore_index=True)

 
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print("df_arrival len after: ",len(df_arrival))
    
    return(df_arrival)

In [32]:
df_rotte = rotte_dataset_simplified[['mmsi','stamp','timestamp','lng','lat','speed']]
df_rotte = df_rotte.sort_values(by=['mmsi','stamp'],ascending=[True,True])
df_rotte = df_rotte.reset_index()

In [33]:
df_arrival= arrival_elaboration(df_rotte)

df_rotte len before:  33474627
0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
Duration: 0:21:46.150702
df_arrival len after:  83187


In [ ]:
#world_porti = pd.read_csv(world_porti_filename, sep=',',low_memory=False)

In [ ]:
#len(world_porti)

In [34]:
ita_porti = world_porti[world_porti['Country']=='ITA']
len(ita_porti)

457

In [35]:
world_porti=world_porti[['Country','Name','Latitude','Longitude','UNLocode']]

In [36]:
world_porti.reset_index(drop=True,inplace=True)

In [37]:
def port_assign(df_rotte,df_porti):
    
    n=len(df_rotte)
    df = df_rotte
    df["port"] = ["null"]*n
    df["port_orig"] = ["null"]*n
    df["UNLocode"] = ["NA"]*n 
    df["UNLocode_orig"] = ["NA"]*n

    start_time = datetime.now()
    
    i=0
    for rotta in df_rotte.itertuples():
        found_orig,found=0,0
        #departure port position
        loc_orig=(float(rotta.lat_orig),float(rotta.lng_orig))
        #arrival port position
        loc=(float(rotta.lat),float(rotta.lng))
        
        for porto in df_porti.itertuples():            
            loc_port=(float(porto.Latitude),float(porto.Longitude))            
            distance1 = hs.haversine(loc,loc_port,unit=Unit.METERS)
            if(distance1 < 5000.0):
                df.loc[i,'port']=porto.Name
                df.loc[i,'UNLocode'] = porto.UNLocode
                found=1
                break
        if (found==0):
            df.loc[i,'port']='Not found'
        
        for porto in df_porti.itertuples():             
            loc_port=(float(porto.Latitude),float(porto.Longitude))
            distance2 = hs.haversine(loc_orig,loc_port,unit=Unit.METERS)
            if(distance2 < 5000.0):
                df.loc[i,'port_orig']=porto.Name
                df.loc[i,'UNLocode_orig'] = porto.UNLocode
                found_orig=1
                break    

        if (found_orig==0):
            df.loc[i,'port_orig']='Not found'
            
        #stampa di controllo
        if(i%10000 == 0):
               print(i)
        i+=1
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print("df_arrival len after: ",len(df_arrival))
        
    return(df)

In [38]:
df_rotte_arrivi_porti = port_assign(df_arrival,world_porti)

0
10000
20000
30000
40000
50000
60000
70000
80000
Duration: 0:48:10.845738
df_arrival len after:  83187


In [39]:
df_rotte_arrivi_porti.head()

,row,mmsi,arrival,departure,lng,lat,lng_orig,lat_orig,speed,status,port,port_orig,UNLocode,UNLocode_orig
0,1,1193046.0,2021-09-01 00:11:25,2021-09-01 00:11:25,8.398606,40.83946,0,0,0,1,Porto Torres,Not found,ITPTO,NA
1,2,1193046.0,2021-09-01 00:28:00,2021-09-01 00:28:00,23.557100,37.96413,8.39861,40.8395,0,1,Keratsini (Agios Georgios),Porto Torres,None,ITPTO
2,3,1193046.0,2021-09-01 00:29:25,2021-09-01 00:29:25,8.398600,40.83945,23.5571,37.9641,0,1,Porto Torres,Keratsini (Agios Georgios),ITPTO,None
3,6,1193046.0,2021-09-01 00:32:02,2021-09-01 00:40:40,23.557080,37.96413,8.3986,40.8394,0,1,Keratsini (Agios Georgios),Porto Torres,None,ITPTO
4,7,1193046.0,2021-09-01 00:41:26,2021-09-01 00:41:26,8.398599,40.83947,23.5571,37.9641,0,1,Porto Torres,Keratsini (Agios Georgios),ITPTO,None


ADD INFORMATION TO THE DATASET

In [26]:
vessels_mmsi_imo_dict = pd.Series(ais_dataset.imo.values,index=ais_dataset.mmsi).to_dict()
#vessels_mmsi_callsign_dict = pd.Series(ais_dataset.callsign.values,index=ais_dataset.mmsi).to_dict()
vessels_mmsi_type_summary_dict = pd.Series(ais_dataset.type_summary.values,index=ais_dataset.mmsi).to_dict()

vessels_mmsi_imo_clean_dict = {k: vessels_mmsi_imo_dict[k] for k in vessels_mmsi_imo_dict if not pd.isna(vessels_mmsi_imo_dict[k])}
#vessels_mmsi_callsign_clean_dict = {k: vessels_mmsi_callsign_dict[k] for k in vessels_mmsi_callsign_dict if not pd.isna(vessels_mmsi_callsign_dict[k])}
vessels_mmsi_type_summary_clean_dict = {k: vessels_mmsi_type_summary_dict[k] for k in vessels_mmsi_type_summary_dict if not pd.isna(vessels_mmsi_type_summary_dict[k])}

In [32]:
vessels_mmsi_type_summary_dict[247046800]
ais_dataset[ais_dataset['mmsi']==247046800]

uniqueValues = (ais_dataset['shiptype'].append(ais_dataset['type_summary'])).unique()
uniqueValues

array([7, 3, 9, 6, 8, 0, 4, 2, 1, 'Cargo', 'Tug', 'Pleasure Craft',
       'NULL', 'Passenger', 'Tanker', 'Special Craft', 'High Speed Craft',
       'Fishing', 'Sailing Vessel', 'Search and Rescue', 'Other',
       'Unspecified'], dtype=object)

In [41]:
df_rotte_arrivi_porti["imo"] = df_rotte_arrivi_porti["mmsi"].astype(int)
df_rotte_arrivi_porti['imo'].replace(vessels_mmsi_imo_clean_dict ,inplace=True)

#df_rotte_arrivi_porti["callsign"] = df_rotte_arrivi_porti["mmsi"].astype(int)
#df_rotte_arrivi_porti['callsign'].replace(vessels_mmsi_callsign_clean_dict ,inplace=True)

df_rotte_arrivi_porti['type_summary'] = df_rotte_arrivi_porti["mmsi"].astype(int)
df_rotte_arrivi_porti['type_summary'].replace(vessels_mmsi_type_summary_clean_dict ,inplace=True)

In [43]:
#export all records
df_rotte_arrivi_porti.to_csv("df_rotte_arrivi_porti_set2021.csv",index=False,sep=',')
#df_rotte_arrivi_porti

In [ ]:
#export only records such that a port has been found
df_rotte_arrivi_porti_world_found = df_rotte_arrivi_porti[(df_rotte_arrivi_porti['port'] != 'Not found')]
#df_rotte_arrivi_porti_world_found.to_csv(rotte_arrivi_porti_world_fname,index=False,sep=',')

COMPUTE STATISTICS OF ARRIVALS IN THE PORTS

In [ ]:
#df_rotte_arrivi_porti_world_found.query("status > 0").groupby(['port']).count().to_csv(arrivi_nei_porti_fname,sep=';')

In [ ]:
#check
#df_rotte_arrivi_porti_world_found[(df_rotte_arrivi_porti_world_found['speed']==0)]

In [ ]:
#df_rotte_all = pd.read_csv(rotte_arrivi_porti_all_fname, sep=',', low_memory=False)

In [ ]:
#df_rotte_passengers_cargo = df_rotte_all[(df_rotte_all['type_summary']=='Passenger') | (df_rotte_all['type_summary']=='Cargo')]


#df_rotte_arrivi_porti_pass_cargo = df_rotte_arrivi_porti[(df_rotte_arrivi_porti['type_summary']=='Passenger') | (df_rotte_arrivi_porti['type_summary']=='Cargo')]
#len(df_rotte_arrivi_porti_pass_cargo)

In [ ]:
#df_rotte_arrivi_porti_pass_cargo.to_csv("DCME\\AIS\\rotte_with_speed_2021\\rotte_arrivi_porti_passengers_cargo_set2021.csv",index=False,sep=',')